# **Primer parcial**

## Universitario: Apaza Villca Cristofer Denilson  

**CU: 35 - 4436  Ing. de Sistemas**



---
## 1.   Importación del **Dataset**







In [70]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




---
## 2.   Carga de Dataset




In [31]:
# ---------- 1. Cargar dataset ----------
# Ajusta el nombre del archivo según tu CSV real
filepath = "C:\IA_PROJECTS\PARCIALES\PARCIAL1\daily-rainfall-at-state-level.csv"
df = pd.read_csv(filepath)

print("Dimensiones:", df.shape)
print("\nPrimeras filas:")
print(df.head())

Dimensiones: (204876, 8)

Primeras filas:
   id        date  state_code        state_name  actual       rfs  normal  \
0   0  2009-01-01           5       Uttarakhand     0.0  0.003906    2.19   
1   1  2009-01-01          18             Assam     0.0  0.000000    0.52   
2   2  2009-01-01          16           Tripura     0.0  0.000000    0.09   
3   3  2009-01-01          36         Telangana     0.0  0.000000    0.17   
4   4  2009-01-01           2  Himachal Pradesh     0.0  0.008566    3.31   

   deviation  
0     -100.0  
1     -100.0  
2     -100.0  
3     -100.0  
4     -100.0  


## Preprocesamiento de datos

In [32]:
# ---------- 2. Revisar datos incompletos ----------
print("\nConteo de valores nulos por columna:")
print(df.isna().sum())

# Revisar tipos de datos
print("\nTipos de datos:")
print(df.dtypes)

# Identificar columnas de texto
text_cols = df.select_dtypes(include=['object']).columns.tolist()
print("\nColumnas de tipo texto:", text_cols)


Conteo de valores nulos por columna:
id                0
date              0
state_code        0
state_name        0
actual        17162
rfs            5865
normal        11518
deviation     31021
dtype: int64

Tipos de datos:
id              int64
date           object
state_code      int64
state_name     object
actual        float64
rfs           float64
normal        float64
deviation     float64
dtype: object

Columnas de tipo texto: ['date', 'state_name']


In [33]:
# ---------- 3. Completar datos incompletos ----------
# Estrategia:
# - numéricas: rellenar con mediana
# - texto: rellenar con valor más frecuente (mode)

for col in df.columns:
    if df[col].isna().sum() > 0:  # solo columnas con NA
        if df[col].dtype in [np.float64, np.int64]:
            mediana = df[col].median()
            df[col] = df[col].fillna(mediana)  # asignación directa
            print(f"Columna {col}: NaN reemplazados con mediana {mediana}")
        else:
            moda = df[col].mode()[0]
            df[col] = df[col].fillna(moda)  # asignación directa
            print(f"Columna {col}: NaN reemplazados con moda '{moda}'")

# Verificación
print("\nNulos restantes después de imputación:")
print(df.isna().sum())



Columna actual: NaN reemplazados con mediana 0.15
Columna rfs: NaN reemplazados con mediana 0.063856
Columna normal: NaN reemplazados con mediana 1.87
Columna deviation: NaN reemplazados con mediana -89.06

Nulos restantes después de imputación:
id            0
date          0
state_code    0
state_name    0
actual        0
rfs           0
normal        0
deviation     0
dtype: int64


In [56]:
# ---------- 4. Convertir texto a números ----------

# 1️⃣ Convertir 'date' a datetime y mantener solo fecha (sin hora)
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    print("Columna 'date' convertida a datetime (solo fecha, sin hora).")

# 2️⃣ Reemplazar 'state_name' por su código numérico
if 'state_name' in df.columns and 'state_code' in df.columns:
    mapping = dict(zip(df['state_name'], df['state_code']))
    df['state_name'] = df['state_name'].map(mapping).astype(int)
    print("\nPrimeras filas después de reemplazar state_name por state_code:")
    print(df[['state_name', 'state_code']].head())

# 3️⃣ Convertir otras columnas de texto a códigos numéricos
for col in df.select_dtypes(include=['object']).columns:
    if col not in ['state_name', 'date']:
        df[col] = df[col].astype('category').cat.codes
        print(f"Columna {col} convertida a códigos numéricos")

# 4️⃣ Verificación final de tipos de datos
print("\nTipos de datos después de conversiones:")
print(df.dtypes)


Columna 'date' convertida a datetime (solo fecha, sin hora).

Primeras filas después de reemplazar state_name por state_code:
   state_name  state_code
0           5           5
1          18          18
2          16          16
3          36          36
4           2           2

Tipos de datos después de conversiones:
id                     int64
date          datetime64[ns]
state_code             int64
state_name             int64
actual               float64
rfs                  float64
normal               float64
deviation            float64
dtype: object


In [57]:
# ---------- 5. Resultado final ----------
print("\nTipos de datos después de transformaciones:")
print(df.dtypes)

# Guardar dataset limpio
df.to_csv("rainfall_clean.csv", index=False)
print("\nArchivo limpio guardado como rainfall_clean.csv")


Tipos de datos después de transformaciones:
id                     int64
date          datetime64[ns]
state_code             int64
state_name             int64
actual               float64
rfs                  float64
normal               float64
deviation            float64
dtype: object

Archivo limpio guardado como rainfall_clean.csv


### Cargar el dataset Limpio

In [58]:
# ---------- 1. Leer el archivo CSV ----------
df_clean = pd.read_csv("rainfall_clean.csv")

In [59]:
# ---------- 2. Revisar dimensiones ----------
print("Dimensiones del dataset original:", df.shape)
print("Dimensiones del dataset leído:", df_clean.shape)

Dimensiones del dataset original: (204876, 8)
Dimensiones del dataset leído: (204876, 8)


In [60]:
# ---------- 3. Verificar tipos de datos ----------
print("\nTipos de datos del dataset original:")
print(df.dtypes)
print("\nTipos de datos del dataset leído desde CSV:")
print(df_clean.dtypes)


Tipos de datos del dataset original:
id                     int64
date          datetime64[ns]
state_code             int64
state_name             int64
actual               float64
rfs                  float64
normal               float64
deviation            float64
dtype: object

Tipos de datos del dataset leído desde CSV:
id              int64
date           object
state_code      int64
state_name      int64
actual        float64
rfs           float64
normal        float64
deviation     float64
dtype: object


In [61]:
# ---------- 4. Revisar si los datos son iguales ----------
# Esto compara si cada celda del DataFrame original y leído es exactamente igual
igualdad = df.equals(df_clean)
print("\n¿El dataset leído es exactamente igual al original?", igualdad)


¿El dataset leído es exactamente igual al original? False


In [62]:
# ---------- 5. Opcional: revisar primeras filas ----------
print("\nPrimeras filas del dataset leído:")
print(df_clean.head())


Primeras filas del dataset leído:
   id        date  state_code  state_name  actual       rfs  normal  deviation
0   0  1970-01-01           5           5     0.0  0.003906    2.19     -100.0
1   1  1970-01-01          18          18     0.0  0.000000    0.52     -100.0
2   2  1970-01-01          16          16     0.0  0.000000    0.09     -100.0
3   3  1970-01-01          36          36     0.0  0.000000    0.17     -100.0
4   4  1970-01-01           2           2     0.0  0.008566    3.31     -100.0


dividimos el dataset

In [65]:
# Asegúrate que el DataFrame esté ordenado por fecha
df.sort_values(by='date', inplace=True)

# Parámetro: porcentaje de datos para entrenamiento
train_ratio = 0.8
train_size = int(len(df) * train_ratio)

# Dividir en train y test
train_df = df.iloc[:train_size].copy()
test_df = df.iloc[train_size:].copy()

# Verificación de tamaños
print(f"Tamaño total: {len(df)}")
print(f"Tamaño train: {len(train_df)}")
print(f"Tamaño test: {len(test_df)}")

# Primeras filas de train y test
print("\nPrimeras filas de train:")
print(train_df.head())
print("\nPrimeras filas de test:")
print(test_df.head())

Tamaño total: 204876
Tamaño train: 163900
Tamaño test: 40976

Primeras filas de train:
            id       date  state_code  state_name  actual         rfs  normal  \
0            0 1970-01-01           5           5    0.00    0.003906    2.19   
136578  136578 1970-01-01          27          27   17.67  185.797279    5.70   
136579  136579 1970-01-01           3           3    0.02    0.033211    5.06   
136580  136580 1970-01-01          20          20    7.71   20.911486    9.16   
136581  136581 1970-01-01          28          28    2.67   14.824959    4.49   

        deviation  
0         -100.00  
136578     210.00  
136579     -99.60  
136580     -15.83  
136581     -40.53  

Primeras filas de test:
          id       date  state_code  state_name  actual        rfs  normal  \
81946  81946 1970-01-01          28          28    0.14   0.771389    1.79   
81947  81947 1970-01-01          12          12   13.88  38.882092   12.39   
81948  81948 1970-01-01          37          37

Normalizamos

In [64]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [66]:
# Columnas a normalizar
cols_to_normalize = ['id','state_code','state_name','actual', 'rfs', 'normal','deviation']

# ---------- 1️⃣ Normalizar train ----------
X_train = train_df[cols_to_normalize].values
X_train_norm, mu, sigma = featureNormalize(X_train)

# Guardamos de nuevo en train_df
train_df[cols_to_normalize] = X_train_norm

# ---------- 2️⃣ Normalizar test usando mu y sigma de train ----------
X_test = test_df[cols_to_normalize].values
X_test_norm = (X_test - mu) / sigma
test_df[cols_to_normalize] = X_test_norm

# ---------- 3️⃣ Verificación ----------
print("Primeras filas normalizadas de train:")
print(train_df.head())

print("\nPrimeras filas normalizadas de test:")
print(test_df.head())


Primeras filas normalizadas de train:
              id       date  state_code  state_name    actual       rfs  \
0      -1.711588 1970-01-01   -1.271899   -1.271899 -0.392637 -0.342007   
136578  0.427839 1970-01-01    0.684717    0.684717  1.504982  6.592618   
136579  0.427855 1970-01-01   -1.449773   -1.449773 -0.390489 -0.340913   
136580  0.427871 1970-01-01    0.062158    0.062158  0.435357  0.438356   
136581  0.427886 1970-01-01    0.773655    0.773655 -0.105900  0.211180   

          normal  deviation  
0      -0.317668  -0.088788  
136578  0.173301   0.137400  
136579  0.083780  -0.088496  
136580  0.657277  -0.027374  
136581  0.004049  -0.045396  

Primeras filas normalizadas de test:
             id       date  state_code  state_name    actual       rfs  \
81946 -0.427944 1970-01-01    0.773655    0.773655 -0.377602 -0.313361   
81947 -0.427928 1970-01-01   -0.649339   -0.649339  1.097966  1.109098   
81948 -0.427913 1970-01-01    1.574088    1.574088  0.089554  0.130789 

## Construccion del modelo

In [73]:
# Número de días que usamos como ventana
window_size = 30

def create_sequences(data, window_size):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

# ---------- 1️⃣ Crear secuencias para train ----------
train_values = train_df['rfs'].values
X_train_seq, y_train_seq = create_sequences(train_values, window_size)

# ---------- 2️⃣ Crear secuencias para test ----------
test_values = test_df['rfs'].values
X_test_seq, y_test_seq = create_sequences(test_values, window_size)

# ---------- 3️⃣ Revisar shapes ----------
print("Shape X_train:", X_train_seq.shape)  # (num_samples, window_size)
print("Shape y_train:", y_train_seq.shape)  # (num_samples,)
print("Shape X_test:", X_test_seq.shape)
print("Shape y_test:", y_test_seq.shape)

# ---------- 4️⃣ Adaptar para LSTM (agregar dimensión de features) ----------
# LSTM espera input de shape (samples, timesteps, features)
X_train_seq = X_train_seq.reshape((X_train_seq.shape[0], X_train_seq.shape[1], 1))
X_test_seq = X_test_seq.reshape((X_test_seq.shape[0], X_test_seq.shape[1], 1))

# ---------- 5️⃣ Convertir a float32 (muy importante para keras LSTM) ----------
X_train_seq = X_train_seq.astype('float32')
y_train_seq = y_train_seq.astype('float32')
X_test_seq = X_test_seq.astype('float32')
y_test_seq = y_test_seq.astype('float32')

print("\nShape X_train listo para LSTM:", X_train_seq.shape)
print("Tipos de datos:")
print(X_train_seq.dtype, y_train_seq.dtype, X_test_seq.dtype, y_test_seq.dtype)


Shape X_train: (163870, 30)
Shape y_train: (163870,)
Shape X_test: (40946, 30)
Shape y_test: (40946,)

Shape X_train listo para LSTM: (163870, 30, 1)
Tipos de datos:
float32 float32 float32 float32


In [76]:
# ---------- Definir modelo ----------
model = Sequential()
model.add(LSTM(50, activation='tanh', input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compilar
model.compile(optimizer='adam', loss='mse')

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_lstm_model.h5', monitor='val_loss', save_best_only=True)

# Entrenar
history = model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop, checkpoint],
    verbose=2
)

# Evaluar
loss_train = model.evaluate(X_train_seq, y_train_seq, verbose=0)
loss_test = model.evaluate(X_test_seq, y_test_seq, verbose=0)
print(f"\nMSE Train: {loss_train:.4f}, MSE Test: {loss_test:.4f}")

# Predicción
y_pred = model.predict(X_test_seq)
print("\nPrimeras 5 predicciones:", y_pred[:5].flatten())


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=best_lstm_model.h5

## Graficar